In [2]:
import os
import numpy
import random
import json
from distutils.dir_util import copy_tree
import shutil
import time

import sys, os
sys.path.append(os.path.dirname('../.'))
from scripts.utils import get_path

In [71]:
path_to_data = '/home/msst/Documents/medtech/data/Inference_IXI'
N_tests = 2
test_size = 3

sample_names = set()
for name in os.listdir(path_to_data):
    if not os.path.isfile(f"{path_to_data}/{name}"):
        sample_names = sample_names.union([name,])


def get_test_train_samples(sample_names, n_test):
    to_test = random.sample(sample_names, k=n_test)
    to_train = list(sample_names.difference(to_test))
    return(to_train, to_test)


testing_info_dict = {}

for i in range(N_tests):
    to_train, to_test = get_test_train_samples(sample_names, test_size)
    testing_info_dict.update({f"test{i+1}" : {'train' : to_train,
                                              'test' : to_test}})

testing_info_dict

{'test1': {'train': ['IXI131',
   'IXI100',
   'IXI057',
   'IXI052',
   'IXI137',
   'IXI020'],
  'test': ['IXI111', 'IXI115', 'IXI077']},
 'test2': {'train': ['IXI131',
   'IXI115',
   'IXI057',
   'IXI077',
   'IXI052',
   'IXI137'],
  'test': ['IXI100', 'IXI111', 'IXI020']}}

In [73]:
fp2 = open(f"{path_to_data}/testing_info.json", 'w')
json.dump(testing_info_dict, sort_keys=True, indent=4, fp=fp2)
fp2.close()

In [72]:
path_to_processed_data = '/home/msst/Documents/medtech/data/HessData_IXI/DATA/processed'
path_to_datasets = '/home/msst/Documents/medtech/data/HessData_IXI_test'

if os.path.exists(path_to_datasets):
    shutil.rmtree(path_to_datasets)
    
if not os.path.exists(path_to_datasets):
    os.mkdir(path_to_datasets)

for test_key in testing_info_dict:
    path_to_set = path_to_datasets+'/'+test_key
    
    if not os.path.exists(path_to_set):
        os.mkdir(path_to_set)
        os.mkdir(path_to_set+"/train")
        os.mkdir(path_to_set+"/test")
        
    
    for name in testing_info_dict[test_key]['train']:
        for tree in get_path(path_to_processed_data, name):
            p = f"{path_to_set}/train/{os.path.basename(tree)}"
            if not os.path.exists(p):
                os.mkdir(p)
            copy_tree(tree, p)

    for name in testing_info_dict[test_key]['test']:
        for tree in get_path(path_to_processed_data, name):
            p = f"{path_to_set}/test/{os.path.basename(tree)}"
            if p[-1]=='0':
                if not os.path.exists(p):
                    os.mkdir(p)
                copy_tree(tree, p)
    

In [3]:
def parse_avg_metrics(avg_metrics_path):
    avg_metrics_dict = json.load(open(avg_metrics_path))
    for model in avg_metrics_dict:
        avg_metrics = {}
        avg_metrics_mask = {}
        print("\nMODEL:", model)
        tests = avg_metrics_dict[model]
        for test in tests:
            data = tests[test]
            for metric in data['avg_metrics']:
                if not avg_metrics.get(metric):
                    avg_metrics.update({metric: [data['avg_metrics'][metric],]})
                else:
                    avg_metrics[metric].append(data['avg_metrics'][metric])
            for metric in data['avg_metrics_mask']:
                if not avg_metrics_mask.get(metric):
                    avg_metrics_mask.update({metric: [data['avg_metrics_mask'][metric],]})
                else:
                    avg_metrics_mask[metric].append(data['avg_metrics_mask'][metric])
        
        
        print("\tavg_metrics:")
        for m in avg_metrics:
            avg_metrics[m] = sum([float(x) for x in avg_metrics[m]])/len(avg_metrics[m])
            print('\t\t', m, ":", avg_metrics[m])
        
        print("\tavg_metrics_mask:")
        for m in avg_metrics_mask:
            avg_metrics_mask[m] = sum([float(x) for x in avg_metrics_mask[m]])/len(avg_metrics_mask[m])
            print('\t\t', m, ":", avg_metrics_mask[m])
            
        #return({"avg_metrics" : avg_metrics, 
        #        "avg_metrics_mask" : avg_metrics_mask})

In [5]:
avg_metrics_path = '/home/msst/Documents/medtech/data/Inference_IXI/out/avg_metrics.json'
_ = parse_avg_metrics(avg_metrics_path)


MODEL: HessNet
	avg_metrics:
		 AVGDIST : 1.4695551666666666
		 DICE : 0.7994174999999999
		 SNSVTY : 0.7802219999999999
	avg_metrics_mask:
		 AVGDIST : 0.4563963333333334
		 DICE : 0.8438953333333334
		 SNSVTY : 0.7850103333333334

MODEL: HessUNet
	avg_metrics:
		 AVGDIST : 0.393968
		 DICE : 0.8877688333333332
		 SNSVTY : 0.8663216666666667
	avg_metrics_mask:
		 AVGDIST : 0.2746206666666666
		 DICE : 0.8942356666666668
		 SNSVTY : 0.86856

MODEL: Unet2d_16ch
	avg_metrics:
		 AVGDIST : 0.8541155
		 DICE : 0.8401875000000001
		 SNSVTY : 0.8431063333333333
	avg_metrics_mask:
		 AVGDIST : 0.3571573333333333
		 DICE : 0.8676181666666666
		 SNSVTY : 0.8533476666666666

MODEL: Unet3d_16ch
	avg_metrics:
		 AVGDIST : 0.3460645
		 DICE : 0.8908043333333334
		 SNSVTY : 0.8903986666666667
	avg_metrics_mask:
		 AVGDIST : 0.2443583333333333
		 DICE : 0.8974291666666667
		 SNSVTY : 0.894417

MODEL: Unet3d_16ch_
	avg_metrics:
		 AVGDIST : 0.42948716666666675
		 DICE : 0.8681536666666667
		 SNSVTY :

In [ ]:
|